此 ipynb 文件只用于内部演示，往 GitHub 提 PR 前会删除。

In [1]:
model_name = 'ced-mini'  # ['ced-tiny', 'ced-mini', 'ced-small', 'ced-base']

In [2]:
# Convert checkpoint from original CED to 🤗 Transformers format
from transformers.models.ced.convert_ced_original_to_pytorch import convert_ced_checkpoint

model_saved_path = f'/home/zhangjunbo/sandbox/{model_name}'
convert_ced_checkpoint(model_name, model_saved_path)

Saving model ced-mini to /home/zhangjunbo/sandbox/ced-mini
No model name specified for CedConfig, use default settings.
Configuration saved in /home/zhangjunbo/sandbox/ced-mini/config.json
Model weights saved in /home/zhangjunbo/sandbox/ced-mini/pytorch_model.bin


In [3]:
import torch
import torchaudio

from transformers import CedConfig, CedForAudioClassification

In [4]:
config = CedConfig(model_name)

DEVICE = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model = CedForAudioClassification(config).from_pretrained(model_saved_path).to(DEVICE).eval()

loading configuration file /home/zhangjunbo/sandbox/ced-mini/config.json
No model name specified for CedConfig, use default settings.
No model name specified for CedConfig, use default settings.
Model config CedConfig {
  "architectures": [
    "CedForAudioClassification"
  ],
  "attn_drop_rate": 0.0,
  "center": true,
  "depth": 12,
  "drop_path_rate": 0.0,
  "drop_rate": 0.0,
  "embed_dim": 256,
  "eval_avg": "mean",
  "f_max": 8000,
  "f_min": 0,
  "freq_patch_out": null,
  "hop_size": 160,
  "init_values": null,
  "mlp_ratio": 4,
  "n_fft": 512,
  "n_mels": 64,
  "num_heads": 4,
  "outputdim": 527,
  "pad_last": true,
  "patch_size": 16,
  "patch_stride": 16,
  "pooling": "mean",
  "qkv_bias": true,
  "target_length": 1012,
  "time_patch_out": null,
  "torch_dtype": "float32",
  "transformers_version": "4.34.0.dev0",
  "win_size": 512
}

loading weights file /home/zhangjunbo/sandbox/ced-mini/pytorch_model.bin
All model checkpoint weights were used when initializing CedForAudioClass

In [5]:
wavpath = '/home/zhangjunbo/src/CED/resources/JeD5V5aaaoI_931_932.wav'
wave, _ = torchaudio.load(wavpath)
wave = wave.to(DEVICE)

output = model(wave).squeeze(0)
output, output.shape

(tensor([4.4041e-04, 1.1244e-06, 3.7792e-08, 1.3794e-06, 1.1796e-09, 6.1123e-08,
         1.2690e-07, 6.3050e-07, 6.6793e-08, 6.9742e-07, 3.3462e-08, 2.5831e-08,
         3.8196e-08, 2.6284e-08, 4.9164e-07, 4.1181e-06, 7.8783e-07, 2.7245e-09,
         2.9222e-08, 3.3038e-08, 3.0569e-09, 9.9292e-08, 6.9381e-10, 2.7488e-10,
         3.5132e-08, 3.0729e-08, 4.4674e-06, 3.2543e-06, 1.2355e-06, 6.2254e-08,
         6.4723e-07, 3.5100e-07, 2.0798e-07, 5.8442e-07, 3.2303e-07, 8.2574e-07,
         2.0684e-07, 4.4407e-07, 1.0994e-06, 7.7535e-06, 1.0299e-04, 5.2003e-06,
         4.1053e-08, 1.0067e-06, 1.9600e-07, 4.6514e-08, 1.4247e-07, 1.7728e-08,
         1.6261e-08, 1.2230e-08, 2.9457e-06, 5.5696e-08, 7.0292e-07, 5.2810e-07,
         4.9031e-05, 2.1013e-06, 2.9222e-10, 3.0011e-07, 2.4952e-07, 8.2705e-08,
         1.9196e-06, 1.6893e-03, 9.0684e-01, 6.1849e-03, 7.0326e-07, 1.1563e-07,
         2.7727e-08, 8.3572e-05, 6.4359e-10, 9.9029e-07, 1.7471e-09, 2.0200e-09,
         1.3394e-03, 4.7462e

In [6]:
import pandas as pd

label_maps = pd.read_csv('http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv').set_index('index')['display_name'].to_dict()

In [7]:
topk = 3

for k, (prob, label) in enumerate(zip(*output.topk(topk))):
    lab_idx = label.item()
    label_name = label_maps[lab_idx]
    print(f"Topk-{k+1} {label_name:<30} {prob:.4f}")

Topk-1 Finger snapping                0.9068
Topk-2 Slap, smack                    0.0610
Topk-3 Inside, small room             0.0426
